# Chapter 13 - Step 0: Generate Demo Data

This notebook corresponds to the data generation step for the Data Quality Agent demo.
It demonstrates:

1. Creating a `sales_transactions` table with synthetic data
2. Injecting a small percentage of data quality issues

**Prerequisites:**
- Run this notebook in a Databricks workspace
- Access to a Unity Catalog catalog and schema

## Configuration

Set the catalog and schema where the demo table will live.
Change these to match your workspace.

In [ ]:
CATALOG = "demo"
SCHEMA  = "finance"
TABLE   = "sales_transactions"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

print(f"Using {CATALOG}.{SCHEMA}")

## Step 1 - Create the sales_transactions table with synthetic data

The table is designed to give the agent interesting columns to reason about:
numeric ranges, date ordering, email formats, enum values, and cross-column
relationships. A small percentage of rows intentionally contain data quality
issues so the agent has real violations to detect.

In [ ]:
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType,
    DoubleType, DateType
)
import random
from datetime import date, timedelta

NUM_ROWS = 2000

random.seed(42)

start_date = date(2024, 1, 1)
statuses = ["completed", "pending", "cancelled", "refunded"]
payment_methods = ["credit_card", "debit_card", "wire_transfer", "paypal"]
domains = ["example.com", "testmail.org", "acme.co", "bigcorp.net"]

def random_email(i):
    return f"customer_{i}@{random.choice(domains)}"

def random_date_pair():
    """Return an (order_date, ship_date) pair where ship >= order most of the time."""
    order = start_date + timedelta(days=random.randint(0, 365))
    # ~5% of rows intentionally have ship_date before order_date
    if random.random() < 0.05:
        ship = order - timedelta(days=random.randint(1, 5))
    else:
        ship = order + timedelta(days=random.randint(0, 14))
    return order, ship

rows = []
for i in range(1, NUM_ROWS + 1):
    original_price = round(random.uniform(5.0, 500.0), 2)
    # ~3% of rows have discount > original (data quality issue)
    if random.random() < 0.03:
        discount_price = round(original_price + random.uniform(1, 50), 2)
    else:
        discount_price = round(original_price * random.uniform(0.5, 1.0), 2)

    quantity = random.randint(1, 200)
    # ~2% negative quantities (data quality issue)
    if random.random() < 0.02:
        quantity = -random.randint(1, 10)

    order_dt, ship_dt = random_date_pair()

    rows.append((
        f"TXN-{i:06d}",
        random_email(i),
        round(original_price * quantity, 2),
        original_price,
        discount_price,
        quantity,
        order_dt,
        ship_dt,
        random.choice(statuses),
        random.choice(payment_methods),
    ))

schema = StructType([
    StructField("transaction_id",  StringType(),  False),
    StructField("customer_email",  StringType(),  True),
    StructField("amount",          DoubleType(),  True),
    StructField("original_price",  DoubleType(),  True),
    StructField("discount_price",  DoubleType(),  True),
    StructField("quantity",        IntegerType(), True),
    StructField("order_date",      DateType(),    True),
    StructField("ship_date",       DateType(),    True),
    StructField("status",          StringType(),  True),
    StructField("payment_method",  StringType(),  True),
])

df = spark.createDataFrame(rows, schema)
df.write.mode("overwrite").saveAsTable(TABLE)

print(f"Created {CATALOG}.{SCHEMA}.{TABLE} with {df.count()} rows")
display(df.limit(10))

## Next Steps

Continue with **01_create_table_and_log_model.py** to log the model and
generate data quality rules.